In [1]:
import monai
import ignite
import os


from prostate158.utils import load_config
from prostate158.train import SegmentationTrainer
# from prostate158.report import ReportGenerator
# from prostate158.viewer import ListViewer
from prostate158.data import segmentation_dataloaders
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# 加载配置文件
config_path = 'config/prostate158/params_search/optimizer/baseline/predict.yaml'
model_pth_path = '/home/data1/skyous/prostate158_log/params_search/3dunet/models/3dunet/3dunet_tverskyloss_0.7868.pt'
config = load_config(config_path) # change to 'tumor.yaml' for tumor segmentation
monai.utils.set_determinism(seed=config.seed)

## 注释一下有关config的数据配置
# config.data.valid_csv = 'southHP_predict/all_v3.csv'
# config.data.train_csv = 'southHP_predict/all_v3.csv'
# config.data.test_csv = 'southHP_predict/all_v3.csv'
# # config.out_dir = 'southHP_predict_config'

# config.data.data_dir = '/home/data1/skyous/SouthHP_prostate/sort_crop'
# config.data.cache_dir = './tmp/monai-cache'

# config.data.valid_csv = '/home/data1/skyous/prostate158/prostate158_test/test.csv'
# print(config.data.valid_csv)

In [19]:
test_loader=segmentation_dataloaders(
            config=config, 
            train=False, 
            valid=False, 
            test=True
        )

print(len(test_loader))

trainer=SegmentationTrainer(
    progress_bar=True, 
    early_stopping = True, 
    metrics = ["MeanDice", "HausdorffDistance", "SurfaceDistance"],
    save_latest_metrics = True,
    config=config
)
trainer.evaluate(checkpoint=model_pth_path, dataloader=test_loader)

/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


19


/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)
/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)
/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.handlers.stats_handler StatsHandler.__init__:name: Current default value of argument `name=None` has been deprecated since version 1.1. It will b

metrics saved to /home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240824_0946_3dunet_1/output


In [7]:
import pandas as pd
import os

output_dir = "/home/data/skyous/prostate158_log/3dunet/anatomyprostate158_params_search20240829_1607_3dunet_1/output"
# output_dir = trainer.config.out_dir
print(output_dir)
# 读取CSV文件
DSC_data = pd.read_csv(os.path.join(output_dir, 'MeanDice_raw.csv'))
HD_data = pd.read_csv(os.path.join(output_dir, 'HausdorffDistance_raw.csv'))
SD_data = pd.read_csv(os.path.join(output_dir, 'SurfaceDistance_raw.csv'))

output_ = '|'

for data in [DSC_data, HD_data, SD_data]:
    # 获取某一列的数值

    for item in ['class0', 'class1', 'mean']:
        column_values = data[item]

        # 计算平均值
        average = column_values.mean()
        if data is DSC_data:
            average = round(average, 4)
        else:
            average = round(average, 2)
        print(item, average)
        output_ += str(average) + '|'

output_ += output_dir + '|'
print(output_)

/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240827_0927_3dunet_1/output
class0 0.857
class1 0.7179
mean 0.7875
class0 20.56
class1 24.73
mean 22.64
class0 2.55
class1 2.16
mean 2.35
|0.857|0.7179|0.7875|20.56|24.73|22.64|2.55|2.16|2.35|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240827_0927_3dunet_1/output|


# 3dunet prelu
|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|3dunet_adam|0.8707|0.7216|0.7961|19.45|26.37|22.91|2.32|2.0|2.16|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240827_0925_3dunet_1/output|
|3dunet_sgd|0.833|0.623|0.728|21.97|31.48|26.72|3.19|2.52|2.85|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240827_0926_3dunet_1/output|
|3dunet_adamW_prelu|0.8763|0.7518|0.814|17.4|23.45|20.43|2.16|1.83|1.99|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240826_0927_3dunet_1/output|
|3dunet_diceloss|0.8575|0.7182|0.7879|20.58|24.73|22.65|2.54|2.14|2.34|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240827_0926_3dunet_2/output|
|3dunet_tverskyloss|0.857|0.7179|0.7875|20.56|24.73|22.64|2.55|2.16|2.35|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240827_0927_3dunet_1/output|





# 3dunet
|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|3dunet_adam|0.8169|0.5878|0.7023|23.37|33.74|28.56|4.21|2.72|3.46|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240824_0935_3dunet_1/output|
|3dunet_adamW|0.8752|0.7437|0.8094|17.97|23.72|20.84|2.12|1.87|2.0|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240824_0938_3dunet_1/output|
|3dunet_sgd|0.8384|0.654|0.7462|19.16|29.47|24.31|3.35|2.47|2.91|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240824_0939_3dunet_1/output|
|3dunet_focalloss|0.8243|0.5514|0.6879|21.56|36.04|28.8|2.95|2.46|2.7|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240824_0941_3dunet_1/output|
|3dunet_diceloss|0.8536|0.7157|0.7846|19.31|24.33|21.82|2.94|2.23|2.58|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240824_0945_3dunet_1/output|
|3dunet_tverskyloss|0.8532|0.7204|0.7868|19.4|24.76|22.08|2.91|2.46|2.68|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240824_0946_3dunet_1/output|
|3dunet_diceCEloss_novograd|0.8582|0.6996|0.7789|21.05|25.95|23.5|2.53|2.11|2.32|/home/data/skyous/prostate158_log/3dunet/anatomyprostate15820240321_2110_3dunet_2/output|
|3dunet_retrain_b1_s4|0.8743|0.7449|0.8096|17.11|23.89|20.5|2.28|1.86|2.07|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240825_0835_3dunet_1/output|
|3dunet_adamW_prelu|0.8763|0.7518|0.814|17.4|23.45|20.43|2.16|1.83|1.99|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240826_0927_3dunet_1/output|

### 一下3dunet均为3dresunet
|network|PZ_dice|TZ_dice|Mean_dice|PZ_HD|TZ_HD|Mean_HD|PZ_ASD|TZ_ASD|Mean_dASD|root|
|--|--|--|--|--|--|--|--|--|--|--|
|3dunet_adamW|0.8856|0.7747|0.8302|17.08|22.19|19.63|1.98|1.74|1.86|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240822_0848_3dunet_1/output|
|3dresunet_novograd|0.8801|0.7624|0.8212|18.2|23.17|20.68|2.1|1.79|1.95|/home/data/skyous/prostate_log/3dunet/anatomyprostate15820240321_2111_3dunet_1/output|
|3dunet_sgd|0.8734|0.7425|0.808|18.65|23.4|21.02|2.19|1.91|2.05|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240822_0854_3dunet_1/output|
|3dunet_diceloss|0.8781|0.7577|0.8179|17.59|23.64|20.62|2.16|1.81|1.99|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240822_0901_3dunet_1/output|
|3dunet_focalloss|0.8675|0.6755|0.7715|18.85|26.4|22.63|2.35|2.02|2.19|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240822_0909_3dunet_1/output|
|3dunet_tverskyloss|0.8756|0.751|0.8133|16.82|23.62|20.22|2.25|1.87|2.06|/home/data1/skyous/prostate158_log/params_search/3dunet/anatomyprostate158_params_search20240822_0910_3dunet_1/output|

|3dunet|0.7712|0.5376|0.6544|22.27|37.2|29.74|4.96|5.33|5.14|southHP_predict/3dunet/output|
|3dresunet|0.7969|0.616|0.7064|20.45|33.91|27.18|4.15|4.93|4.54|southHP_predict/3dresunet/anatomyprostate15820240322_0923_3dunet_1/output|
|nnunet|0.752|0.5074|0.6297|24.47|39.01|31.74|5.56|6.95|6.26|/home/skyous/git/prostate158/southHP_predict/nnformer/anatomyprostate15820240322_0927_nnformer_1/output|
